In [ ]:
from google.colab import drive
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.color import rgb2gray
from skimage.transform import resize
from scipy import signal
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
import zipfile

zip_file = "/content/train.zip"
extract_folder = "/content/trainimages"
csv_file = "/content/trainLabels.csv"

if not os.path.exists(extract_folder):
    os.makedirs(extract_folder)

with zipfile.ZipFile(zip_file, 'r') as archive:
    archive.extractall(extract_folder)

df = pd.read_csv(csv_file)
image_folder = "/content/trainimages/train"

def load_image(image_id, img_size=(32, 32)):
    img_path = os.path.join(image_folder, f"{image_id}.png")
    if os.path.exists(img_path):
        img = imread(img_path)
        img_resized = resize(img, img_size)
        img_gray = rgb2gray(img_resized)
        return img_gray
    else:
        print(f"Image {image_id}.png not found!")
        return None

X = [0,1,2,3,4,5,6,0,1,2,3,4,5,6,0,0,0]
H_L = [0.05, 0.2, 0.5, 0.2, 0.05]
H_H = [-1, 2, -1]

y_low = np.convolve(X, H_L, mode='same')
y_high = np.convolve(X, H_H, mode='same')

plt.figure(figsize=(10,4))
plt.plot(X, label="Original Signal", linestyle="dashed")
plt.plot(y_low, label="Low-pass Filtered")
plt.plot(y_high, label="High-pass Filtered")
plt.legend()
plt.title("1D Convolution with Filters")
plt.show()

sample_id = df["id"][0]
img = load_image(sample_id)

fil1 = np.array([[ 0, -1,  0], [-1, 4, -1], [ 0, -1,  0]])
fil2 = np.array([[ 0.2, 0.5,  0.2], [0.5, 1, 0.5], [0.2, 0.5, 0.2]])
fil3 = np.ones((5,5)) / 25

grad1 = signal.convolve2d(img, fil1, boundary='symm', mode='same')
grad2 = signal.convolve2d(img, fil2, boundary='symm', mode='same')
grad3 = signal.convolve2d(img, fil3, boundary='symm', mode='same')

fig, axes = plt.subplots(1, 4, figsize=(15,5))
axes[0].imshow(img, cmap="gray"); axes[0].set_title("Original")
axes[1].imshow(abs(grad1), cmap="gray"); axes[1].set_title("Edge Detection")
axes[2].imshow(grad2, cmap="gray"); axes[2].set_title("Sharpened")
axes[3].imshow(grad3, cmap="gray"); axes[3].set_title("Smoothed")
plt.show()

image_size = (32, 32)
X_data = []
y_data = []

for index, row in df.iterrows():
    img = load_image(row["id"])
    if img is not None:
        X_data.append(img)
        y_data.append(row["label"])

X_data = np.array(X_data).reshape(len(X_data), 32, 32, 1)
y_data = np.array(pd.factorize(y_data)[0])

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

num_classes = len(np.unique(y_train))
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

model = Sequential([
    Flatten(input_shape=(32, 32, 1)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title("Training vs Validation Loss")
plt.show()

y_pred = np.argmax(model.predict(X_test), axis=1)
y_true = np.argmax(y_test, axis=1)

conf_matrix = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8,6))
sns.heatmap(conf_matrix, annot=True, cmap="Blues", fmt="d")
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()
##